<center> <h1>Assignment #3</h1> </center>
<center> <h2>DSP</h2> </center>


In [ ]:
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
import os
from os import listdir
import numpy as np 
from matplotlib import pyplot as plt
import pandas as pd    
from scipy.fft import dct
from sklearn.decomposition import PCA
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from scipy.fft import dct
from sklearn.decomposition import PCA
import numpy as np
from sklearn.cluster import KMeans
import numpy as np
from sklearn import mixture
from sklearn import svm
from skimage.feature import hog
from skimage import data, exposure
import cv2
import tensorflow as tf
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten,AvaragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import time
#%%
x_tst=[]
y_tst=[]                      
TST_PATH = r'E:\engineering\Learning\EECE\fourth year\second term\dsp\projects\assignment2\Reduced MNIST Data\Reduced Testing data'
for digit in listdir(TST_PATH):
    for ex in listdir(TST_PATH+'\\'+digit):
        x_tst.append(imread(TST_PATH+'\\'+digit+'\\'+ex))
        y_tst.append(digit)
x_tst=np.asarray(x_tst)
y_tst=np.asarray(y_tst)

x_train=[]
y_train=[]
TRAIN_PATH = r'E:\engineering\Learning\EECE\fourth year\second term\dsp\projects\assignment2\Reduced MNIST Data\Reduced Trainging data'
for digit in listdir(TRAIN_PATH):
    for ex in listdir(TRAIN_PATH+'\\'+digit):
        x_train.append(imread(TRAIN_PATH+'\\'+digit+'\\'+ex))
        y_train.append(digit)
#x_train=np.asarray(x_train)
#y_train=np.asarray(y_train)
#print(x_train.shape,y_train.shape) 
#print(x_tst.shape,y_tst.shape)

# train=pd.DataFrame({'img':x_train,'label':y_train})
# train=train.sample(frac=1).reset_index(drop=True)
# x_train=np.stack(np.asarray(train['img']))
# y_train=np.asarray(train['label'])

x_train=np.asarray(x_train).reshape((10000,28,28))
print(x_train.shape) 
x_tst=np.asarray(x_tst).reshape((2000,28,28))
print(x_tst.shape)
y_train=np.asarray(y_train).astype(int)
y_tst=np.asarray(y_tst).astype(int)

#%%


""" DCT Feature extraction"""

dct_train_data = dct(dct(x_train, axis=1), axis=2)
dct_tst_data = dct(dct(x_tst, axis=1), axis=2)
print(dct_train_data.shape)
print(dct_tst_data.shape)



rows=28
columns=28
dct_train_data_compressed=np.empty((10000,180))
solution=[]
for k in range(10000):    
    solution=[]
    for i in range(rows):
        for j in range(columns):
            if((i+j) < 19):
                solution.append(dct_train_data[k, i, j])
    dct_train_data_compressed[k]=np.asarray(solution[:-10])
    
print(dct_train_data_compressed.shape)

dct_tst_data_compressed=np.empty((2000,180))
for k in range(2000):    
    solution=[]
    for i in range(rows):
        for j in range(columns):
            if((i+j) < 19):
                solution.append(dct_tst_data[k, i, j])
    dct_tst_data_compressed[k]=np.asarray(solution[:-10])
print(dct_tst_data_compressed.shape)
x_train_dct=dct_train_data_compressed
x_tst_dct=dct_tst_data_compressed

""" PCA Feature extraction"""
pca_breast = PCA(n_components=270)
x_train_pca = pca_breast.fit_transform(x_train.reshape((10000,784)))
x_tst_pca = pca_breast.transform(x_tst.reshape((2000,784)))
sum(pca_breast.explained_variance_ratio_)

#%%
def HOG_(images):
    feature_array=[]
    for image in images:
        ret, bw_img = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
        features, hog_img = hog(bw_img, orientations=8, pixels_per_cell=(6, 6), visualize=True, multichannel=False)
       # features = exposure.rescale_intensity(features, in_range=(0, 10))
        #features=np.where(features.reshape([28*28,])==1)[0]
        feature_array.append(features)
    
    # list_len = [len(i) for i in feature_array]
    # max_len=(max(list_len))
    
    # for i,ex in enumerate(feature_array):
    #    feature_array[i]=np.append(ex,np.zeros(max_len-len(ex)))
   
    feature_array=np.asarray(feature_array)

    return feature_array
x_tst_edge=HOG_(x_tst)
x_train_edge=HOG_(x_train)

#%%

def KM(X,Y,Xt,Yt,n):
    
    t = time.time()

    kmeans = KMeans(n_clusters=n, random_state=0).fit(X)
    t_elapsed=time.time()-t
    classes=kmeans.labels_
    
    # print([sum(classes==i) for i in range(n)])
    # print(sum([sum(classes==i) for i in range(n)]))
    
    
    
    labels = np.zeros_like(classes)
    translation=np.zeros((n,1))
    for i in range(n):
        mask = (classes == i)
        labels[mask] = mode(Y[mask])[0]
        translation[i]=mode(Y[mask])[0]
    
    
    
    
    prediction =kmeans.predict(Xt)
    for i in range(len(prediction)):
        prediction[i]=translation[prediction[i]]
    
    
    accuarcy=accuracy_score(prediction, Yt)
    train_accuarcy=accuracy_score(labels, Y)
    
    return train_accuarcy,accuarcy,t_elapsed
    
 #%%
[acc_KM_10_pca_train,acc_KM_10_pca_tst,time_KM_10_pca_train]=KM(x_train_pca,y_train,x_tst_pca,y_tst,10) 
[acc_KM_40_pca_train,acc_KM_40_pca_tst,time_KM_40_pca_train]=KM(x_train_pca,y_train,x_tst_pca,y_tst,40)
[acc_KM_160_pca_train,acc_KM_160_pca_tst,time_KM_160_pca_train]=KM(x_train_pca,y_train,x_tst_pca,y_tst,160)

    
#%%
[acc_KM_10_dct_train,acc_KM_10_dct_tst,time_KM_10_dct_train]=KM(x_train_dct,y_train,x_tst_dct,y_tst,10) 
[acc_KM_40_dct_train,acc_KM_40_dct_tst,time_KM_40_dct_train]=KM(x_train_dct,y_train,x_tst_dct,y_tst,40)
[acc_KM_160_dct_train,acc_KM_160_dct_tst,time_KM_160_dct_train]=KM(x_train_dct,y_train,x_tst_dct,y_tst,160)

#%%

[acc_KM_10_edge_train,acc_KM_10_edge_tst,time_KM_10_edge_train]=KM(x_train_edge,y_train,x_tst_edge,y_tst,10) 
[acc_KM_40_edge_train,acc_KM_40_edge_tst,time_KM_40_edge_train]=KM(x_train_edge,y_train,x_tst_edge,y_tst,40) 
[acc_KM_160_edge_train,acc_KM_160_edge_tst,time_KM_160_edge_train]=KM(x_train_edge,y_train,x_tst_edge,y_tst,160) 

#%%
print('KM 10')
print('[pca_train, pca_tst,Time_elapsed]',[acc_KM_10_pca_train,acc_KM_10_pca_tst,time_KM_10_pca_train])
print('[dct_train, dct_tst,Time_elapsed]',[acc_KM_10_dct_train,acc_KM_10_dct_tst,time_KM_40_pca_train])
print('[hog_train, hog_tst,Time_elapsed]',[acc_KM_10_edge_train,acc_KM_10_edge_tst,time_KM_160_pca_train])

print('KM 40')
print('[pca_train, pca_tst,Time_elapsed]',[acc_KM_40_pca_train,acc_KM_40_pca_tst,time_KM_10_dct_train])
print('[dct_train, dct_tst,Time_elapsed]',[acc_KM_40_dct_train,acc_KM_40_dct_tst,time_KM_40_dct_train])
print('[hog_train, hog_tst,Time_elapsed]',[acc_KM_40_edge_train,acc_KM_40_edge_tst,time_KM_160_dct_train])

print('KM 160')
print('[pca_train, pca_tst,Time_elapsed]',[acc_KM_160_pca_train,acc_KM_160_pca_tst,time_KM_10_edge_train])
print('[dct_train, dct_tst,Time_elapsed]',[acc_KM_160_dct_train,acc_KM_160_dct_tst,time_KM_40_edge_train])
print('[hog_train, hog_tst,Time_elapsed]',[acc_KM_160_edge_train,acc_KM_160_edge_tst,time_KM_160_edge_train])


#%%
def GM(X,Y,Xt,Yt,n):
    
    t = time.time()

    gmm = mixture.GaussianMixture(n_components=n).fit(X)
    t_elapsed=time.time()-t

    classes=gmm.predict(X)
    
    # print([sum(classes==i) for i in range(n)])
    # print(sum([sum(classes==i) for i in range(n)]))
    
    
    
    labels = np.zeros_like(classes)
    translation=np.zeros((n,1))
    for i in range(n):
        mask = (classes == i)
        labels[mask] = mode(Y[mask])[0]
        translation[i]=mode(Y[mask])[0]
    
    
    
    
    prediction =gmm.predict(Xt)
    for i in range(len(prediction)):
        prediction[i]=translation[prediction[i]]
    
    
    accuarcy=accuracy_score(prediction, Yt)
    train_accuarcy=accuracy_score(labels, Y)
    
    return train_accuarcy,accuarcy,t_elapsed
#%%
[acc_GMM_10_pca_train,acc_GMM_10_pca_tst,time_GMM_10_pca_train]=GM(x_train_pca,y_train,x_tst_pca,y_tst,10) 
[acc_GMM_40_pca_train,acc_GMM_40_pca_tst,time_GMM_40_pca_train]=GM(x_train_pca,y_train,x_tst_pca,y_tst,40)
[acc_GMM_160_pca_train,acc_GMM_160_pca_tst,time_GMM_160_pca_train]=GM(x_train_pca,y_train,x_tst_pca,y_tst,160)

    
#%%

[acc_GMM_10_dct_train,acc_GMM_10_dct_tst,time_GMM_10_dct_train]=GM(x_train_dct,y_train,x_tst_dct,y_tst,10) 
[acc_GMM_40_dct_train,acc_GMM_40_dct_tst,time_GMM_40_dct_train]=GM(x_train_dct,y_train,x_tst_dct,y_tst,40)
[acc_GMM_160_dct_train,acc_GMM_160_dct_tst,time_GMM_160_dct_train]=GM(x_train_dct,y_train,x_tst_dct,y_tst,160)

#%%

[acc_GMM_10_edge_train,acc_GMM_10_edge_tst,time_GMM_10_edge_train]=GM(x_train_edge,y_train,x_tst_edge,y_tst,10)
[acc_GMM_40_edge_train,acc_GMM_40_edge_tst,time_GMM_40_edge_train]=GM(x_train_edge,y_train,x_tst_edge,y_tst,40) 
[acc_GMM_160_edge_train,acc_GMM_160_edge_tst,time_GMM_160_edge_train]=GM(x_train_edge,y_train,x_tst_edge,y_tst,160) 
#%%
print('GMM 10')
print('[pca_train, pca_tst,Time_elapsed]',[acc_GMM_10_pca_train,acc_GMM_10_pca_tst,time_GMM_10_pca_train])
print('[dct_train, dct_tst,Time_elapsed]',[acc_GMM_10_dct_train,acc_GMM_10_dct_tst,time_GMM_40_pca_train])
print('[hog_train, hog_tst,Time_elapsed]',[acc_GMM_10_edge_train,acc_GMM_10_edge_tst,time_GMM_160_pca_train])

print('GMM 40')
print('[pca_train, pca_tst,Time_elapsed]',[acc_GMM_40_pca_train,acc_GMM_40_pca_tst,time_GMM_10_dct_train])
print('[dct_train, dct_tst,Time_elapsed]',[acc_GMM_40_dct_train,acc_GMM_40_dct_tst,time_GMM_40_dct_train])
print('[hog_train, hog_tst,Time_elapsed]',[acc_GMM_40_edge_train,acc_GMM_40_edge_tst,time_GMM_160_dct_train])

print('GMM 160')
print('[pca_train, pca_tst,Time_elapsed]',[acc_GMM_160_pca_train,acc_GMM_160_pca_tst,time_GMM_10_edge_train])
print('[dct_train, dct_tst,Time_elapsed]',[acc_GMM_160_dct_train,acc_GMM_160_dct_tst,time_GMM_40_edge_train])
print('[hog_train, hog_tst,Time_elapsed]',[acc_GMM_160_edge_train,acc_GMM_160_edge_tst,time_GMM_160_edge_train])



#%%
def SVM(X,Y,Xt,Yt,k):

    clf = svm.SVC(kernel=k) # Linear Kernel
    t = time.time()
    clf.fit(X, Y)
    t_elapsed=time.time()-t

    labels = clf.predict(X)
    y_pred = clf.predict(Xt)
        
    train_accuarcy = accuracy_score(Y, labels)
    accuarcy = accuracy_score(Yt, y_pred)

    return train_accuarcy,accuarcy,t_elapsed

#%%
[acc_SVM_pca_train,acc_SVM_pca_tst,time_SVM_pca_train]=SVM(x_train_pca,y_train,x_tst_pca,y_tst,'Linear') 
[acc_SVM_dct_train,acc_SVM_dct_tst,time_SVM_dct_train]=SVM(x_train_dct,y_train,x_tst_dct,y_tst,'Linear') 
[acc_SVM_edge_train,acc_SVM_edge_tst,time_SVM_edge_train]=SVM(x_train_edge,y_train,x_tst_edge,y_tst,'Linear') 
print('SVM_Linear')
print('[pca_train, pca_tst , time_elapsed]',[acc_SVM_pca_train,acc_SVM_pca_tst,time_SVM_pca_train])
print('[dct_train, dct_tst , time_elapsed]',[acc_SVM_dct_train,acc_SVM_dct_tst,time_SVM_dct_train])
print('[hog_train, hog_tst , time_elapsed]',[acc_SVM_edge_train,acc_SVM_edge_tst,time_SVM_edge_train])

#%%
[acc_SVM_pca_train_,acc_SVM_pca_tst_,time_SVM_pca_train_]=SVM(x_train_pca,y_train,x_tst_pca,y_tst,'rbf') 
[acc_SVM_dct_train_,acc_SVM_dct_tst_,time_SVM_dct_train_]=SVM(x_train_dct,y_train,x_tst_dct,y_tst,'rbf') 
[acc_SVM_edge_train_,acc_SVM_edge_tst_,time_SVM_edge_train_]=SVM(x_train_edge,y_train,x_tst_edge,y_tst,'rbf') 
print('SVM_RPF')
print('[pca_train, pca_tst , time_elapsed]',[acc_SVM_pca_train_,acc_SVM_pca_tst_,time_SVM_pca_train_])
print('[dct_train, dct_tst , time_elapsed]',[acc_SVM_dct_train_,acc_SVM_dct_tst_,time_SVM_dct_train_])
print('[hog_train, hog_tst , time_elapsed]',[acc_SVM_edge_train_,acc_SVM_edge_tst_,time_SVM_edge_train_])
#%%

leNet=tf.keras.models.Sequential()
leNet.add(Conv2D(filters=8, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
leNet.add(AvaragePooling2D(strides=2))
leNet.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='relu'))
leNet.add(AvaragePooling2D(strides=2))
leNet.add(Flatten())
leNet.add(Dense(256, activation='relu'))
leNet.add(Dense(84, activation='relu'))
leNet.add(Dense(10, activation='softmax'))
